In [24]:
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        # add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower())
         for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(
        word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['What', 'is', 'a', 'coronavirus'], 'What is a coronavirus?'), (['What', 'is', 'covid', '19'], 'What is COVID-19?'), (['symptoms', 'of', 'covid', '19'], 'What are the symptoms of COVID 19?'), (['symptoms', 'of', 'coronavirus'], 'What are the symptoms of COVID 19?'), (['covid', '19', 'symptoms'], 'What are the symptoms of COVID 19?'), (['coronavirus', 'symptoms'], 'What are the symptoms of COVID 19?'), (['How', 'does', 'COVID', '19', 'spread'], 'How does COVID 19 spread?'), (['How', 'does', 'coronavirus', 'spread'], 'How does COVID 19 spread?'), (['What', 'can', 'I', 'do', 'to', 'protect', 'myself', 'and', 'prevent', 'th

<ipython-input-24-615119979a25>:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


10/10 [==============================] - 0s 1ms/step - loss: 2.9247 - accuracy: 0.0426
Epoch 2/200
10/10 [==============================] - 0s 998us/step - loss: 2.8092 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.6772 - accuracy: 0.2128
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 2.5652 - accuracy: 0.3191
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 2.5164 - accuracy: 0.1702
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 2.3294 - accuracy: 0.4043
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 2.2134 - accuracy: 0.2979
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0861 - accuracy: 0.4043
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7854 - accuracy: 0.5319
Epoch 10/200
10/10 [==============================] - 0s 1ms/step - loss: 1.8159 - accuracy: 0.5319
Epoch 11/200
10/10 

10/10 [==============================] - 0s 883us/step - loss: 0.2162 - accuracy: 0.9149
Epoch 83/200
10/10 [==============================] - 0s 887us/step - loss: 0.0721 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 997us/step - loss: 0.1228 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 997us/step - loss: 0.0630 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0550 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 887us/step - loss: 0.1985 - accuracy: 0.9362
Epoch 88/200
10/10 [==============================] - 0s 887us/step - loss: 0.1149 - accuracy: 0.9574
Epoch 89/200
10/10 [==============================] - 0s 881us/step - loss: 0.0841 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 838us/step - loss: 0.2097 - accuracy: 0.9362
Epoch 91/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1135 - accuracy: 0.97

10/10 [==============================] - 0s 1ms/step - loss: 0.0998 - accuracy: 0.9574
Epoch 163/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1248 - accuracy: 0.9362
Epoch 164/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0308 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0721 - accuracy: 0.9787
Epoch 166/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0661 - accuracy: 0.9787
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0409 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0319 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 997us/step - loss: 0.0449 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0979 - accuracy: 0.9574
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0517 - accuracy: 0.9574
Ep

In [25]:
import os
import random
import json
from keras.models import load_model
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [26]:
nltk.download('omw-1.4')
model = load_model(os.path.join( 'chatbot_model.h5'))
intents = json.loads(
    open(os.path.join( 'intents.json')).read())
words = pickle.load(
    open(os.path.join(  'words.pkl'), 'rb'))
classes = pickle.load(
    open(os.path.join(  'classes.pkl'), 'rb'))

print('words', words)
print('classes', classes)


def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                #if show_details:
                    #print("found in bag: %s" % word)
    #print("np.array(bag)", np.array(bag))
    return(np.array(bag))


def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words, show_details=True)
    flag = False
    return_list = []
    for x in p:
        if x == 1:
            flag = True
    if (flag == True):
        res = model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
        # sorting strength probability
        results.sort(key=lambda x: x[1], reverse=True)

        #print("result", results)
        for r in results:
            return_list.append(
                {"intent": classes[r[0]], "probability": str(r[1])})
    else:
        return_list.append({"intent": "noanswer", "probability": str(1)})

    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg)
    #print('ints', ints)
    res = getResponse(ints, intents)
    #print('res', res)
    return res

words ["'s", '19', 'a', 'about', 'after', 'am', 'and', 'antibiotic', 'any', 'anyone', 'are', 'awesome', 'by', 'bye', 'can', 'catch', 'chatting', 'coronavirus', 'covid', 'covid-19', 'cure', 'disease', 'do', 'doe', 'drug', 'effect', 'effected', 'effective', 'everyone', 'for', 'get', 'goodbye', 'having', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'incubation', 'is', 'last', 'later', 'likely', 'long', 'me', 'measure', 'medicine', 'myself', 'next', 'nice', 'of', 'or', 'period', 'positive', 'prevent', 'preventing', 'protect', 'protection', 'see', 'should', 'side', 'spread', 'surface', 'symptom', 'test', 'thank', 'thanks', 'that', 'the', 'therapy', 'there', 'till', 'time', 'to', 'touching', 'treating', 'treatment', 'vaccine', 'what', 'worry', 'you']
classes ['Are antibiotics effective in preventing or treating the COVID-19', 'Are there any medicines or therapies that can prevent or cure COVID-19', 'How does COVID 19 spread?', 'How likely am I to catch COVID-19?', 'H

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Meme\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [40]:
# test chatbot 
from termcolor import colored
print ("CHAT START (BLANK AND ENTER TO STOP)")
try:
    while (1):
        x=input(colored("YOU:",'red'))
        if (x.strip()==""):
            break
        print ( colored("CHATBOT:",'blue'), colored(chatbot_response(x),'green'))
except:
    print("ERROR")

print("CHAT END")


CHAT START (BLANK AND ENTER TO STOP)
YOU:hello
CHATBOT: Hi there
YOU:what is coronavirus? 
CHATBOT: Coronaviruses are a large family of viruses which may cause illness in animals or humans.  In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19.
YOU:what are coronavirus symptoms?
CHATBOT: The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually. Some people become infected but donÃ¢â‚¬â„¢t develop any symptoms and don't feel unwell. Most people (about 80%) recover from the disease without needing special treatment. Around 1 out of every 6 people who gets COVID-19 becomes seriously ill